In [ ]:
import numpy as np
import scipy.linalg as spla
import scipy.sparse as sps
import matplotlib.pyplot as plt

from pymor.models.iosys import LTIModel

In [ ]:
plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize'] = [9., 6.]

# Building an LTIModel

## Heat equation over a steel rod

In this exercise we consider [heat flow in a uniform rod](https://en.wikipedia.org/wiki/Heat_equation#Heat_flow_in_a_uniform_rod). In this model the temperature average over the entire rod as well as the temperature on the left-hand side are measured. Additionally, the heat flux on the left-hand side of the rod can be controlled by an input $u(t)$.

$$
\begin{align*}
  \partial_t T(\xi, t) & = \alpha \partial_{\xi\xi} T(\xi, t), & \xi \in (0, 1),\ t > 0, \\
  \partial_{\xi} T(0, t) & = T(0, t) - u(t), & t > 0, \\
  \partial_{\xi} T(1, t) & = -T(0, t), & t > 0, \\
  y_1(t) & = T(0, t), & t > 0, \\
  y_2(t) & = \int_0^1 T(\xi, t) \,\mathrm{d}\xi, & t > 0.
\end{align*}
$$

$\alpha = 1.172 \times 10^{-5} \frac{\text{m}^2}{\text{s}}$

## Central difference discretization

We consider $n$ equidistant grid points for a central difference discretization:

$0 = \xi_1 < \xi_2 < \ldots < \xi_n = 1$,
$h = \frac{1}{n - 1}$,
$\xi_k = (k - 1) h$,
$$
\begin{align*}
  \dot{x}_k(t) & = \alpha \frac{x_{k - 1}(t) - 2 x_k(t) + x_{k + 1}(t)}{h^2}, & k = 1, 2, \ldots, n, \\
  \frac{x_2(t) - x_0(t)}{2 h} & = x_1(t) - u(t), \\
  \frac{x_{n + 1}(t) - x_{n - 1}(t)}{2 h} & = -x_n(t), \\
  y_1(t) & = x_1(t), \\
  y_2(t) & = \frac{1}{n} \sum_{k = 1}^n x_k(t).
\end{align*}
$$

## Simplification

$$
\begin{align*}
  \frac{1}{2} \dot{x}_1(t) & = \alpha \frac{-(1 + h) x_1(t) + x_2(t)}{h^2} + \frac{\alpha}{h} u(t), \\
  \dot{x}_k(t) & = \alpha \frac{x_{k - 1}(t) - 2 x_k(t) + x_{k + 1}(t)}{h^2}, & k = 2, 3, \ldots, n - 1, \\
  \frac{1}{2} \dot{x}_n(t) & = \alpha \frac{x_{n - 1}(t) - (1 + h) x_n(t)}{h^2}, \\
  y_1(t) & = x_1(t), \\
  y_2(t) & = \frac{1}{n} \sum_{k = 1}^n x_k(t).
\end{align*}
$$

In [ ]:
n = 100

E = sps.eye(n, format='lil')
E[0, 0] = E[-1, -1] = 0.5
E = E.tocsc()

alpha = 1.172e-5
c = alpha * (n - 1)**2
A = sps.diags([(n - 1) * [c], n * [-2 * c], (n - 1) * [c]], [-1, 0, 1], format='lil')
A[0, 0] = -alpha * (n - 1) * n
A[-1, -1] = -alpha * (n - 1) * n
A = A.tocsc()

B = np.zeros((n, 1))
B[0, 0] = alpha * (n - 1)

C = np.zeros((2, n))
C[0, -1] = 1
C[1, :] = 1/n

In [ ]:
fom = LTIModel.from_matrices(A, B, C, E=E)

In [ ]:
fom

In [ ]:
print(fom)

## Poles

In [ ]:
poles_fom = fom.poles()
fig, ax = plt.subplots()
_ = ax.plot(poles_fom.real, poles_fom.imag, '.')

## Bode plot

In [ ]:
w = np.logspace(-8, 1, 500)
fig, ax = plt.subplots(4, 1, squeeze=False, figsize=(10, 8), tight_layout=True)
_ = fom.bode_plot(w, ax=ax)

## Magnitude plot

In [ ]:
fig, ax = plt.subplots()
_ = fom.mag_plot(w, ax=ax)

## Hankel singular values

In [ ]:
hsv = fom.hsv()
fig, ax = plt.subplots()
_ = ax.semilogy(range(1, n + 1), hsv, '.-')

## $\mathcal{H}_2$ norm

In [ ]:
fom.h2_norm()

## $\mathcal{H}_\infty$ norm

In [ ]:
fom.hinf_norm()

# Balanced Truncation

In [ ]:
from pymor.reductors.bt import BTReductor

In [ ]:
bt = BTReductor(fom)

In [ ]:
rom_bt = bt.reduce(tol=1e-5)

In [ ]:
r = 10
rom_bt = bt.reduce(r)

In [ ]:
err_bt = fom - rom_bt

## Poles

In [ ]:
poles_bt = rom_bt.poles()
fig, ax = plt.subplots()
_ = ax.plot(poles_bt.real, poles_bt.imag, '.')

## Error magnitude plot

In [ ]:
fig, ax = plt.subplots()
_ = err_bt.mag_plot(w, ax=ax)

## Relative $\mathcal{H}_2$ error

In [ ]:
err_bt.h2_norm() / fom.h2_norm()

## Relative $\mathcal{H}_\infty$ error

In [ ]:
err_bt.hinf_norm() / fom.hinf_norm()

# IRKA

In [ ]:
from pymor.reductors.h2 import IRKAReductor

In [ ]:
irka = IRKAReductor(fom)

In [ ]:
r = 10
rom_irka = irka.reduce(r)

In [ ]:
err_irka = fom - rom_irka

## Poles

In [ ]:
poles_irka = rom_irka.poles()
fig, ax = plt.subplots()
_ = ax.plot(poles_irka.real, poles_irka.imag, '.')

## Error magnitude plot

In [ ]:
fig, ax = plt.subplots()
_ = err_irka.mag_plot(w, ax=ax)

## Relative $\mathcal{H}_2$ error

In [ ]:
err_irka.h2_norm() / fom.h2_norm()

## Relative $\mathcal{H}_\infty$ error

In [ ]:
err_irka.hinf_norm() / fom.hinf_norm()